In [1]:
import cv2
import numpy as np
import threading
import time
import multiprocessing


In [2]:
def detect_cheating_phone():
    net_phone = cv2.dnn.readNetFromDarknet("yolov4-tiny.cfg", "yolov4-tiny.weights")
    classes_phone = open("coco.names", "r").read().strip().split("\n")
    cap_phone = cv2.VideoCapture(0)

    while True:
        ret, frame = cap_phone.read()

        if not ret:
            break

        height, width = frame.shape[:2]

        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
        net_phone.setInput(blob)

        output_layers_names = net_phone.getUnconnectedOutLayersNames()
        layer_outputs = net_phone.forward(output_layers_names)

        boxes = []
        confidences = []
        class_ids = []

        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)

                if classes_phone[class_id] == "cell phone":
                    confidence = scores[class_id]

                    if confidence > 0.5:
                        center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype(int)
                        x, y = int(center_x - w / 2), int(center_y - h / 2)
                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

        indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        for i in range(len(boxes)):
            x, y, w, h = boxes[i]
            class_id = class_ids[i]
            label = "cheating using phone"
            confidence = confidences[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (128, 0, 128), 2)
            cv2.putText(frame, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (128, 0, 128), 2)

        cv2.imshow("Cheating Detection (Phone)", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap_phone.release()
    cv2.destroyAllWindows()


In [3]:
def detect_cheating_person():
    labelsPath = 'cheatingPersonModel/labels.names'
    LABELS = open(labelsPath).read().strip().split("\n")
    COLORS = np.array([[143, 0, 255], [220, 180, 61]])

    net_person = cv2.dnn.readNetFromDarknet('cheatingPersonModel/cheatingModel.cfg', 'cheatingPersonModel/cheatingModel.weights')
    ln_person = net_person.getLayerNames()
    ln_person = [ln_person[i - 1] for i in net_person.getUnconnectedOutLayers()]

    W, H = None, None
    cam_person = cv2.VideoCapture(0)

    while True:
        grabbed, frame = cam_person.read()

        if not grabbed:
            break

        if not W or not H:
            (H, W) = frame.shape[:2]

        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        net_person.setInput(blob)
        layerOutputs = net_person.forward(ln_person)

        boxes = []
        confidences = []
        classIDs = []

        for output in layerOutputs:
            for detection in output:
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]

                if confidence > 0.4:
                    box = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)

        idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)

        if len(idxs) > 0:
            for i in idxs.flatten():
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])
                color = [int(c) for c in COLORS[classIDs[i]]]
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
                cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        cv2.imshow('Cheating Detection (Person)', frame)

        if cv2.waitKey(1) == ord('q'):
            break

    cam_person.release()
    cv2.destroyAllWindows()

In [4]:
if __name__ == "__main__":
    # Start a thread for each detection task
    thread_phone = threading.Thread(target=detect_cheating_phone)
    thread_person = threading.Thread(target=detect_cheating_person)

    thread_phone.start()
    thread_person.start()

    thread_phone.join()
    thread_person.join()

## Models combined in one window

In [35]:
import cv2
import numpy as np
import time


In [36]:

def perform_phone_detection(frame, net_phone, classes_phone):
    height, width = frame.shape[:2]

    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net_phone.setInput(blob)

    output_layers_names = net_phone.getUnconnectedOutLayersNames()
    layer_outputs = net_phone.forward(output_layers_names)

    boxes = []
    confidences = []
    class_ids = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)

            if classes_phone[class_id] == "cell phone":
                confidence = scores[class_id]

                if confidence > 0.5:
                    center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype(int)
                    x, y = int(center_x - w / 2), int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    return boxes, confidences, class_ids


In [37]:

def draw_phone_detection(frame, boxes, confidences, class_ids):
    for i in range(len(boxes)):
        x, y, w, h = boxes[i]
        class_id = class_ids[i]
        label = "cheating using phone"
        confidence = confidences[i]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (128, 0, 128), 2)
        cv2.putText(frame, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (128, 0, 128), 2)


In [38]:

def perform_person_detection(frame, net_person,ln_person, LABELS):
    (H, W) = frame.shape[:2]

    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net_person.setInput(blob)
    layerOutputs = net_person.forward(ln_person)

    boxes = []
    confidences = []
    classIDs = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if confidence > 0.4:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)

    return boxes, confidences, classIDs


In [39]:

def draw_person_detection(frame, boxes, confidences, classIDs, COLORS, LABELS):
    if len(classIDs) > 0:  
        for i in range(len(classIDs)): 
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
            cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [40]:

net_phone = cv2.dnn.readNetFromDarknet("yolov4-tiny.cfg", "yolov4-tiny.weights")
classes_phone = open("coco.names", "r").read().strip().split("\n")

labelsPath = 'cheatingPersonModel/labels.names'
LABELS = open(labelsPath).read().strip().split("\n")
COLORS = np.array([[143, 0, 255], [220, 180, 61]])

net_person = cv2.dnn.readNetFromDarknet('cheatingPersonModel/cheatingModel.cfg', 'cheatingPersonModel/cheatingModel.weights')
ln_person = net_person.getLayerNames()
ln_person = [ln_person[i - 1] for i in net_person.getUnconnectedOutLayers()]

cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()

    if not ret:
        break
    frame = cv2.resize(frame, (800, 600))

    phone_boxes, phone_confidences, phone_class_ids = perform_phone_detection(frame, net_phone, classes_phone)
    draw_phone_detection(frame, phone_boxes, phone_confidences, phone_class_ids)

    person_boxes, person_confidences, person_class_ids = perform_person_detection(frame, net_person,ln_person, LABELS)
    draw_person_detection(frame, person_boxes, person_confidences, person_class_ids, COLORS, LABELS)

    cv2.imshow("Combined Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()


In [28]:
cam.release()
cv2.destroyAllWindows()